In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
water_quality = pd.read_csv("water_quality_master_file.csv")

water_quality

,monitoringlocationidentifier,monitoringlocationname,latitudemeasure,longitudemeasure,activitystartdate,characteristicname,result_value,units,mcl_value,mcl_units,result_mg_l,exceedance_flag,is_proxy_unit
0,GLEC-10124,Colorado River,33.258610,-114.672240,11/11/2023,Arsenic,0.0027,mg/L,0.01,mg/l,0.0027,0,NaN
1,SOBOBA_WQX-IC2,Indian Creek 2nd Crossing,33.463970,-116.509600,16/11/2023,Arsenic,4.1000,ug/L,0.01,mg/l,0.0041,0,NaN
2,GLEC-10124,Colorado River,33.258610,-114.672240,14/09/2023,Arsenic,0.0031,mg/L,0.01,mg/l,0.0031,0,NaN
3,USGS-10260950,WF MOJAVE R AB MOJAVE R FORKS RES NR HESPERIA CA,34.338892,-117.257821,26/10/2023,Arsenic,1.9000,ug/L,0.01,mg/l,0.0019,0,NaN
4,USGS-11074000,SANTA ANA R BL PRADO DAM CA,33.883349,-117.645330,30/10/2023,Arsenic,3.0000,ug/L,0.01,mg/l,0.0030,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5291,MORONGO1_WQX-M003,Millard 003,33.961460,-116.800000,19/04/2023,Selenium,2.5000,ug/L,0.05,mg/l,0.0025,0,NaN
5292,PPNPROGRAM-SS7,Pond #1,39.550000,-123.438400,12/01/2023,Selenium,0.0100,mg/L,0.05,mg/l,0.0100,0,NaN
5293,GLEC-10124,Colorado River,33.258610,-114.672240,14/09/2023,Selenium,0.0015,mg/L,0.05,mg/l,0.0015,0,NaN
5294,21NEV1_WQX-C8,West Fork Carson River @ Paynesville,38.808781,-119.777199,05/12/2023,Selenium,2.5000,ug/L,0.05,mg/l,0.0025,0,NaN


In [3]:
water_quality = water_quality.dropna(subset="monitoringlocationname")

In [4]:
water_quality

,monitoringlocationidentifier,monitoringlocationname,latitudemeasure,longitudemeasure,activitystartdate,characteristicname,result_value,units,mcl_value,mcl_units,result_mg_l,exceedance_flag,is_proxy_unit
0,GLEC-10124,Colorado River,33.258610,-114.672240,11/11/2023,Arsenic,0.0027,mg/L,0.01,mg/l,0.0027,0,NaN
1,SOBOBA_WQX-IC2,Indian Creek 2nd Crossing,33.463970,-116.509600,16/11/2023,Arsenic,4.1000,ug/L,0.01,mg/l,0.0041,0,NaN
2,GLEC-10124,Colorado River,33.258610,-114.672240,14/09/2023,Arsenic,0.0031,mg/L,0.01,mg/l,0.0031,0,NaN
3,USGS-10260950,WF MOJAVE R AB MOJAVE R FORKS RES NR HESPERIA CA,34.338892,-117.257821,26/10/2023,Arsenic,1.9000,ug/L,0.01,mg/l,0.0019,0,NaN
4,USGS-11074000,SANTA ANA R BL PRADO DAM CA,33.883349,-117.645330,30/10/2023,Arsenic,3.0000,ug/L,0.01,mg/l,0.0030,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5291,MORONGO1_WQX-M003,Millard 003,33.961460,-116.800000,19/04/2023,Selenium,2.5000,ug/L,0.05,mg/l,0.0025,0,NaN
5292,PPNPROGRAM-SS7,Pond #1,39.550000,-123.438400,12/01/2023,Selenium,0.0100,mg/L,0.05,mg/l,0.0100,0,NaN
5293,GLEC-10124,Colorado River,33.258610,-114.672240,14/09/2023,Selenium,0.0015,mg/L,0.05,mg/l,0.0015,0,NaN
5294,21NEV1_WQX-C8,West Fork Carson River @ Paynesville,38.808781,-119.777199,05/12/2023,Selenium,2.5000,ug/L,0.05,mg/l,0.0025,0,NaN


In [5]:
station_summary = water_quality.groupby(["monitoringlocationidentifier", "characteristicname"]).agg(
    total_samples = ("result_mg_l", "count"),
    compliance_failures = ("exceedance_flag", "sum"),
    peak_result = ("result_mg_l", "max"),
    mcl_value = ("mcl_value", "first")
).reset_index()

station_summary

,monitoringlocationidentifier,characteristicname,total_samples,compliance_failures,peak_result,mcl_value
0,11NPSWRD_WQX-SFAN_I&M_BG2,Nitrate,13,0,1.50,10.0
1,11NPSWRD_WQX-SFAN_I&M_BG2,Turbidity,13,2,11.30,5.0
2,11NPSWRD_WQX-SFAN_I&M_BJC1,Nitrate,13,0,0.41,10.0
3,11NPSWRD_WQX-SFAN_I&M_BJC1,Turbidity,13,0,4.34,5.0
4,11NPSWRD_WQX-SFAN_I&M_CDC2,Nitrate,5,0,0.54,10.0
...,...,...,...,...,...,...
803,USGS-391448122565601,Turbidity,7,6,520.00,5.0
804,YUROKTEP_WQX-TC,Turbidity,16,8,27.56,5.0
805,YUROKTEP_WQX-TG,Turbidity,19,8,36.00,5.0
806,YUROKTEP_WQX-TR,Turbidity,24,6,43.10,5.0


In [6]:
station_summary["exceedance_rate"] = round(((station_summary["compliance_failures"]/station_summary["total_samples"])*100), 2)
station_summary["severity_ratio"] = round((station_summary["peak_result"]/station_summary["mcl_value"]), 2)

problem_stations = station_summary[station_summary["compliance_failures"] > 0].sort_values(
    by=["characteristicname", "severity_ratio"], ascending=[True, False]
)

problem_stations

,monitoringlocationidentifier,characteristicname,total_samples,compliance_failures,peak_result,mcl_value,exceedance_rate,severity_ratio
613,USGS-10257720,Arsenic,1,1,0.050,0.01,100.00,5.00
389,OVIWC-LP-SS-3,Arsenic,1,1,0.034,0.01,100.00,3.40
382,OVIWC-LP-SS-2,Arsenic,1,1,0.033,0.01,100.00,3.30
78,21NEV1_WQX-EFS,Arsenic,20,7,0.023,0.01,35.00,2.30
483,SIR_WQX-SW-WBC-LR2,Arsenic,4,1,0.014,0.01,25.00,1.40
...,...,...,...,...,...,...,...,...
629,USGS-10336780,Turbidity,4,2,5.900,5.00,50.00,1.18
574,TRINIDADRANCHERIAEPA-MH-1,Turbidity,11,1,5.700,5.00,9.09,1.14
134,COCOPAH_INDIAN-CUP30,Turbidity,2,2,5.600,5.00,100.00,1.12
547,SYCEO_WQX-ZDC-9,Turbidity,1,1,5.200,5.00,100.00,1.04


In [7]:
problems_sorted = problem_stations.sort_values(
    by=['characteristicname', 'compliance_failures', 'exceedance_rate'],
    ascending=[True, False, False]
)

top_offenders = problems_sorted.groupby('characteristicname').head(10)

top_offenders

,monitoringlocationidentifier,characteristicname,total_samples,compliance_failures,peak_result,mcl_value,exceedance_rate,severity_ratio
78,21NEV1_WQX-EFS,Arsenic,20,7,0.02300,0.010,35.00,2.30
613,USGS-10257720,Arsenic,1,1,0.05000,0.010,100.00,5.00
389,OVIWC-LP-SS-3,Arsenic,1,1,0.03400,0.010,100.00,3.40
382,OVIWC-LP-SS-2,Arsenic,1,1,0.03300,0.010,100.00,3.30
116,BVR-BVRST01,Arsenic,3,1,0.01100,0.010,33.33,1.10
483,SIR_WQX-SW-WBC-LR2,Arsenic,4,1,0.01400,0.010,25.00,1.40
718,USGS-324329114430001,Copper,1,1,5.00000,1.300,100.00,3.85
118,BVR-BVRST01,Lead,3,3,0.02500,0.015,100.00,1.67
125,BVR-BVRST02,Lead,2,2,0.02500,0.015,100.00,1.67
655,USGS-11173200,Lead,26,2,0.02190,0.015,7.69,1.46


In [8]:
top_offenders["monitoringlocationidentifier"].value_counts()

monitoringlocationidentifier
BVR-BVRST01                     2
USGS-11173200                   2
21NEV1_WQX-EFS                  1
OVIWC-LP-SS-3                   1
USGS-10257720                   1
OVIWC-LP-SS-2                   1
SIR_WQX-SW-WBC-LR2              1
USGS-324329114430001            1
BVR-BVRST02                     1
OVIWC-LP-SS-5                   1
PPNPROGRAM-SS0                  1
PPNPROGRAM-SS2                  1
PPNPROGRAM-SS8                  1
PPNPROGRAM-SS9                  1
MORONGO1_WQX-BC1                1
MORONGO1_WQX-M003               1
USGS-390030122502101            1
USGS-11449820                   1
USGS-391057122544301            1
USGS-11449235                   1
USGS-11449255                   1
USGS-11449370                   1
USGS-09427520                   1
SYCEO_WQX-ZDC-7                 1
SYCEO_WQX-ZDC-3                 1
SYCEO_WQX-ZDC-5                 1
TEPA29-SW-3                     1
TEPA29-SW-1D                    1
COYOTEVAL_WQX-CABFO

In [9]:
final_export = pd.merge(
    station_summary,
    water_quality[['monitoringlocationidentifier', 'monitoringlocationname', 'latitudemeasure', 'longitudemeasure']],
    on='monitoringlocationidentifier',
    how='left'
)

def categorize_severity(ratio):
    if ratio > 5: return "Critical"
    elif ratio > 1: return "Exceedant"
    else: return "Compliant"

final_export['status'] = final_export['severity_ratio'].apply(categorize_severity)

final_export.to_csv("C:/Users/Liam Dujon/Desktop/Data Science/Projects/Water Quality Project/summary_data.csv", index=False)